In [49]:
import pandas as pd
import warnings
import sqlite3
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
warnings.filterwarnings("ignore")
from datetime import datetime

In [61]:
df = pd.read_clipboard()

In [52]:
df.fillna(0, inplace=True)

In [57]:
result = df.groupby(['Group', 'No.1', '規格 ', '花紋', '產地']).sum().stack().reset_index(name="volume")

In [60]:
result.columns = ['Group', 'PCODE', '規格 ', '花紋', '產地', '月份', 'volume']

In [63]:
map_dict = dict(zip(df["商品代號"], df["分類"]))

In [65]:
result["Premium"] = result["PCODE"].map(map_dict)

In [67]:
result.to_clipboard()

In [2]:
# Build connection
con = sqlite3.connect(r"C:\Users\kc.hsu\Python workspace\BS_Spider\web_scraping\web_scraping\scraping_df")

In [3]:
# Build cursor
cur = con.cursor()

In [4]:
# Get all data from the table "bsmi"
script = "select * from bsmi"

In [5]:
# Read data from sql with pandas
df = pd.read_sql(script, con)

In [6]:
df.shape

(127058, 11)

In [7]:
df = df.drop_duplicates()

In [8]:
df.shape

(127058, 11)

In [12]:
result = df.copy()

In [13]:
result["extract_size"] = result["size"].str.extract(r"(\d{1,3}.?/?\s?\d{0,3} ?[R|Z|ZR|RF]+ ?[\d\.]+)")
result["extract_size_2"] = result["model"].str.extract(r"(\d{1,3}.?/?\s?\d{0,3} ?[R|Z|ZR|RF|F]+ ?[\d\.]+)")
result["final_size"] = result.apply(lambda x: x["extract_size"] if pd.notna(x["extract_size"]) else x["model"] if pd.notna(x["model"]) else "na", axis=1)
result["final_size"] = result["final_size"].str.replace("Z", "").str.replace(" ", "")
filter_list = ["BRIDGESTONE", "R150", "R150II", "R294", "R168", "M729", "R162", "R102", "R118"]
result["bs1"] = result["en_product_name"].str.contains("BRIDGESTONE|R150|R150II|R294|R168|M729|R162")
result["bs2"] = result["size"].str.contains("BRIDGESTONE|R150|R150II|R294|R168|M729|R162")
result["bs3"] = result["model"].str.contains("BRIDGESTONE|R150|R150II|R294|R168|M729|R162")
result["Date"] = pd.to_datetime(result["month"].map(lambda x: str(int(x[:3]) + 1911) + x[3:]))

In [23]:
result["Verification"] = result[["bs1", "bs2", "bs3"]].apply(tuple, axis=1)

In [27]:
result["BS"] = result["Verification"].map(lambda x: "BS" if True in x else "No")

In [30]:
result = result[['Date', 'company_name', 'tyre_type', 'category', 'upper_tag',
       'lower_tag', 'cn_product_name', 'en_product_name', 'items', 'size',
       'model', 'final_size', "BS"]]

In [41]:
result = result[result["Date"] >= "2021-01-01"]

In [42]:
today = datetime.today() \
        .strftime("%Y%m%d")
export_path = "D:/kc.hsu\OneDrive - Bridgestone/數據/標準檢驗局/" + today + "_商檢局資料.xlsx"
result.to_excel(export_path, index=False)